In [27]:
from src.resolve_init import GetResolve
from pprint import pprint

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()


def get_subfolder_by_name(subfolder_name: str):
    all_subfolder = root_folder.GetSubFolderList()
    subfolder_dict = {
        subfolder.GetName(): subfolder for subfolder in all_subfolder
    }
    return subfolder_dict.get(subfolder_name, "")


# 拿到 Source 文件夹下所有卷的 video clip 信息，作为 MediaPoolItem


def get_video_clips_in_date_group_source_folder(date_group: str) -> list:
    source_clip_list = []

    date_group_folder_structure = get_subfolder_by_name(
        date_group
    ).GetSubFolderList()  # type: ignore

    for folder in date_group_folder_structure:
        if folder.GetName() == "Source":
            for each_video_reel in folder.GetSubFolderList():
                reel_clips = each_video_reel.GetClipList()
                for clip in reel_clips:
                    source_clip_list.append(clip)

    return source_clip_list


def create_timeline(timeline_name: str, width: int, height: int):
        media_pool.CreateEmptyTimeline(timeline_name)
        current_timeline = project.GetCurrentTimeline()
        current_timeline.SetSetting("useCustomSettings", "1")
        current_timeline.SetSetting("timelineResolutionWidth", str(width))
        current_timeline.SetSetting("timelineResolutionHeight", str(height))
        return current_timeline.SetSetting("timelineFrameRate", str(24))

sorted_clip_name = [
    clip
    for clip in get_video_clips_in_date_group_source_folder("DAY_015_20230421")
    if clip.GetClipProperty("Scene") == "10"
]
media_pool.AppendToTimeline(sorted_clip_name)

In [29]:
from src.resolve_init import GetResolve
from pprint import pprint

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()

current_timeline.GetSetting()

{'timelineDropFrameTimecode': '0',
 'timelineFrameRate': '24',
 'timelineInputResMismatchBehavior': 'scaleToCrop',
 'timelineInterlaceProcessing': '0',
 'timelineOutputPixelAspectRatio': 'square',
 'timelineOutputResMatchTimelineRes': '1',
 'timelineOutputResMismatchBehavior': 'scaleToFit',
 'timelineOutputResolutionHeight': '1080',
 'timelineOutputResolutionWidth': '1998',
 'timelinePixelAspectRatio': 'square',
 'timelineResolutionHeight': '1080',
 'timelineResolutionWidth': '1998',
 'useCustomSettings': '1',
 'videoDataLevels': 'Video',
 'videoDataLevelsRetainSubblockAndSuperWhiteData': '0',
 'videoMonitorBitDepth': '10',
 'videoMonitorFormat': 'HD 1080p 24',
 'videoMonitorMatrixOverrideFor422SDI': 'Rec.601',
 'videoMonitorSDIConfiguration': 'dual_link',
 'videoMonitorScaling': 'Bilinear',
 'videoMonitorUse444SDI': '0',
 'videoMonitorUseHDROverHDMI': '0',
 'videoMonitorUseLevelA': '0',
 'videoMonitorUseMatrixOverrideFor422SDI': '0',
 'videoMonitorUseStereoSDI': '0',
 'superScale': 1}